# Load

In [1]:
import scipy.ndimage.filters as filters

import math
import matplotlib.pyplot as plt
import skimage.data
import os
import lasagne
import theano
import theano.tensor as T
import numpy as np
from mnist import main
from load import loadNetwork
from load import leertxt

# Nombre archivo de fruta

In [2]:
nombre = '/home/jorge/Dropbox/AGTS/LectorImagenes/modelo.txt'

# Iniciar carga

In [3]:
Depth,  Width, Drop_in, Drop_hid, Box_size = leertxt(nombre)


In [4]:
Parameters = str(Depth) +','+ str(Width) +','+ str(Drop_in) +','+ str(Drop_hid) +','+ str(Box_size)
input_var = T.tensor4('inputs')
target_var = T.ivector('targets')

In [5]:
network=main('custom_cnn:'+ Parameters, input_var, target_var, 1)

Loading data...
Building model and compiling functions...
Starting training...
Epoch 1 of 1 took 0.034s
  training loss:		0.723872
  validation loss:		0.658467
  validation accuracy:		87.50 %
Final results:
  test loss:			0.704270
  test accuracy:		39.71 %


# Cargar valores de la red entrenada

In [6]:
network= loadNetwork(network, 'prueba')


# Correr el analysis

## Cargar imagen

In [ ]:
root = os.path.abspath('dataset/')
im_path    = root + '/tree1.jpg'

img = skimage.data.load(im_path)
img = img.mean(axis=2)

## Construir funcion

In [ ]:
val_prediction = lasagne.layers.get_output(network, deterministic=True)
eval_fn = theano.function([input_var], val_prediction)
output = np.zeros((img.shape[0], img.shape[1], 2))

## Sliding window

In [ ]:

margin = int(math.floor(Box_size / 2))
for x in xrange(margin, int(img.shape[0] - margin)):
    for y in xrange(margin, int(img.shape[1]-margin)):
        patch = img[x-margin:x+margin+1, y-margin:y+margin+1]
        patch = patch.reshape(1,1, Box_size, Box_size)
        output[x,y,:] = eval_fn(patch)

## Detect Local Maxima

In [ ]:
# Smooth the response map
heatmap = filters.gaussian_filter(output[:,:,1], 3) 
# Thresholding
seg = heatmap>0.9
# Non-maximum suppression
detections = np.where(np.multiply(seg, heatmap == filters.maximum_filter(heatmap, 3)))

print len(detections[0])

## Mostrar imagen

In [ ]:
fig, ax = plt.subplots()
ax.imshow(img, interpolation='nearest', cmap=plt.cm.gray)
plt.plot(detections[1], detections[0], 'b.', markersize=12, mew=3)

    
ax.axis('image')
ax.set_xticks([])
ax.set_yticks([])
plt.show()
plt.rcParams['figure.figsize'] = 20,20